In [ ]:
import tensorflow_hub as hub
from tensorflow import keras
import nltk
import re
import numpy as np

In [ ]:
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words("english")
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")
clf = keras.models.load_model('model84.h5')

In [ ]:
def preprocess(text):
    text = re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
          tokens.append(token)
    return " ".join(tokens)

def encoding(tweet):
    tweet = preprocess(tweet)
    tokens = tweet.split(" ")
    embedding = []
    for token in tokens:
        embedding.append(embed([token]).numpy()[0])

    embedding = np.array(embedding)
    padded_tweet = padding(embedding)
    return padded_tweet

def padding(encoded_tweet):
    count = 70 - encoded_tweet.shape[0]
    pad = np.zeros((1,250))
    for i in range(count):
        encoded_tweet = np.concatenate((pad,encoded_tweet), axis=0)
    return encoded_tweet

def processed_tweets(tweet):
    x = []
    enc = encoding(tweet)
    x.append(enc)
    x = np.array(x)
    return x

def predict(tweet, model):
    prediction = model.predict(processed_tweets(tweet))[0][0]
    result =  1.0 if prediction > 0.5 else 0.0
    return result

In [ ]:
'''
Execute all cells above this once

call predict function to get sentiment

input type :-
    tweet - string
    model - keras model

return type :- 
    float

refer the example below
'''

predict('very good government', clf)